# Use of Measured Aerosol Optical Depth and Precipitable Water to Model Clear Sky Irradiance
*presented at: 2017 IEEE PVSC-44*
*by: Mark A. Mikofski, Clifford W. Hansen, William F. Holmgren and Gregory M. Kimball*

## Analysis
Predicting irradiance is crucial for developing solar power systems. Clear sky models are used to predict irradiance based on solar position and atmospheric data. This paper compares clear sky predictions using atmospheric data from ECMWF with irradiance measurments from SURFRAD. This notebook presents the analysis for the paper and presentation given by the authors at the 2017 IEEE PVSC-44 in Washingtion DC June 25-30th.

### ECMWF
The European Center for Medium Weather Forecast hosts datasets for the EU Copernicus MACC clearsky model. This data set contains aerosol optical depth and total column water vapor data derived from ground measurments and satelite data for the entire globe from 2003 to 2012.

### SURFRAD
SURFRAD data can be viewed online or downloaded from a NOAA FTP site <ftp://aftp.cmdl.noaa.gov/data/radiation/surfrad/>.

In [ ]:
# imports
from datetime import datetime
from ftplib import FTP
import logging
from StringIO import StringIO

import h5py
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

# set figure size
sns.set_context('notebook', rc={'figure.figsize': (16, 8)})

# logging
logging.basicConfig()
LOGGER = logging.getLogger('pvsc44-clearsky-aod')
LOGGER.setLevel(logging.DEBUG)


In [ ]:
# Get SURFRAD data from NOAA

TESTING = True  # flag for testing
ECMWF_MACC_START = 2003
ECMWF_MACC_STOP = 2012
NOAA_FTP = 'aftp.cmdl.noaa.gov'
SURFRAD_PATH = '/'.join(['data', 'radiation', 'surfrad'])
SURFRAD_SITES = {'Bondville_IL': 'bon', 'Boulder_CO': 'tbl', 'Desert_Rock_NV': 'dra', 'Fort_Peck_MT': 'fpk',
                 'Goodwin_Creek_MS': 'gwn', 'Penn_State_PA': 'psu', 'Sioux_Falls_SD': 'sxf'}
# these sites have data outside of the time range of the atmospheric measurements
TOO_SOON = 'Alamosa_CO', 'Red_Lake_AZ', 'Rutland_VT', 'Wasco_OR'
DTYPES = [int, int, int, int, int, int, float, float, float, int, float, int, float, int, float, int, float, int, float, int,
          float, int, float, int, float, int, float, int, float, int, float, int, float, int, float, int, float, int, float, int,
          float, int, float, int, float, int, float, int]
NAMES = ['year', 'jday', 'month', 'day', 'hour', 'min', 'dt', 'zen', 'dw_solar', 'qc_dwsolar', 'uw_solar', 'qc_uwsolar',
         'direct_n', 'qc_direct_n', 'diffuse', 'qc_diffuse', 'dw_ir', 'qc_dwir', 'dw_casetemp', 'qc_dwcasetemp',
         'dw_dometemp', 'qc_dwdometemp', 'uw_ir', 'qc_uwir', 'uw_casetemp', 'qc_uwcasetemp', 'uw_dometemp', 'qc_uwdometemp',
         'uvb', 'qc_uvb', 'par', 'qc_par', 'netsolar', 'qc_netsolar', 'netir', 'qc_netir', 'totalnet', 'qc_totalnet',
         'temp', 'qc_temp', 'rh', 'qc_rh', 'windspd', 'qc_windspd', 'winddir', 'qc_winddir', 'pressure', 'qc_pressure']

noaa_ftp_conn = FTP(NOAA_FTP)  # connection
noaa_ftp_conn.connect()  # if timedout
noaa_ftp_conn.login()  # as anonymous
noaa_ftp_conn.cwd(SURFRAD_PATH)  # navigate to surfrad folder

# loop over sites
data = {}
stations = {}
for surfrad_site, station_id in SURFRAD_SITES.iteritems():
    noaa_ftp_conn.cwd(surfrad_site)  # open site folder
    years = []  # get list of available years
    noaa_ftp_conn.retrlines('NLST', lambda _: years.append(_))
    # loop over yearly site data
    for y in years:
        # skip any non-year items in the folder
        try:
            year = int(y)
        except ValueError:
            continue
        # limit to data overlapping ECMWF atmospheric data (for now)
        if ECMWF_MACC_START <= year <= ECMWF_MACC_STOP:
            noaa_ftp_conn.cwd(y)  # open site folder
            files = []  # get list of available daily datafiles
            noaa_ftp_conn.retrlines('NLST', lambda _: files.append(_))
            for f in files:
                buf = StringIO()  # buffer is a Python builtin, **SURPRISE!**
                noaa_ftp_conn.retrbinary('RETR %s' % f, buf.write)
                buf.seek(0)
                station_name = buf.readline().strip()
                latitude, longitude, elevation, _ = buf.readline().split(None, 3)
                if station_id not in stations:
                    stations[station_id] = [(year, f, station_name, float(latitude), float(longitude), float(elevation))]
                else:
                    stations[station_id].append((year, f, station_name, float(latitude), float(longitude), float(elevation)))
                df = pd.read_table(buf, names=NAMES, dtype=zip(NAMES, DTYPES), delim_whitespace=True,
                                   parse_dates=[['year', 'month', 'day', 'hour', 'min']],
                                   date_parser=lambda y, mo, d, h, m: datetime(y, mo, d, h, m),
                                   na_values='-9999.9', index_col='year_month_day_hour_min')
                if station_id not in data:
                    data[station_id] = df
                else:
                    data[station_id] = data[station_id].append(df)
            noaa_ftp_conn.cwd('..')  # open site folder
            start, stop = ('%d-01-01T00:00:00' % year), ('%d-12-31T23:59:59' % year)
            data_array = data[station_id][start:stop].to_records()
            data_array = data_array.astype(np.dtype(
                [('year_month_day_hour_min', str, 30)]
                + [(str(n), d) for n, d in data_array.dtype.descr if n != 'year_month_day_hour_min']
            ))
            try:
                with h5py.File('%s-%d.h5' % (station_id, year)) as h5f: h5f['data'] = data_array
            except RuntimeError:
                pass
    LOGGER.debug('station: %s', station_name)

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)
Use of Measured Aerosol Optical Depth and Precipitable Water to Model Clear Sky Irradiance by [Mark A. Mikofski, Clifford W. Hansen, William F. Holmgren and Gregory M. Kimball](https://github.com/mikofski/pvsc44-clearsky-aod) is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).


In [ ]:
data['sxf'][['zen', 'dw_solar', 'direct_n', 'diffuse', 'temp', 'rh', 'windspd', 'winddir', 'pressure']]['2003-06-15':'2003-06-16']

In [ ]:
data['sxf'][['direct_n', 'diffuse']]['2003-06-15':'2003-06-16'].plot()